### Graph Node Classification

In [4]:
from utils import *
import os.path as osp
import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv


In [5]:
dataset = Planetoid(root="data/Cora", name="Cora")

In [6]:
dataset[0], dataset[0].keys

(Data(edge_index=[2, 10556], test_mask=[2708], train_mask=[2708], val_mask=[2708], x=[2708, 1433], y=[2708]),
 ['x', 'edge_index', 'y', 'train_mask', 'val_mask', 'test_mask'])

In [7]:
dataset.num_classes, dataset[0].x, dataset[0].edge_index, dataset[0].num_nodes, dataset[0].num_edges, dataset[0].num_node_features, dataset[0].contains_isolated_nodes(), dataset[0].contains_self_loops(), dataset[0].is_directed()

(7,
 tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]),
 tensor([[   0,    0,    0,  ..., 2707, 2707, 2707],
         [ 633, 1862, 2582,  ...,  598, 1473, 2706]]),
 2708,
 10556,
 1433,
 False,
 False,
 False)

In [8]:
dataset[0].train_mask.sum().item(),dataset[0].val_mask.sum().item(),dataset[0].test_mask.sum().item()

(140, 500, 1000)

In [9]:
class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(300):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    if epoch<=20:
        print("Epoch:",epoch," Loss:",loss)
    optimizer.step()

Epoch: 0  Loss: tensor(1.9459, device='cuda:0', grad_fn=<NllLossBackward>)
Epoch: 1  Loss: tensor(1.8367, device='cuda:0', grad_fn=<NllLossBackward>)
Epoch: 2  Loss: tensor(1.7206, device='cuda:0', grad_fn=<NllLossBackward>)
Epoch: 3  Loss: tensor(1.5555, device='cuda:0', grad_fn=<NllLossBackward>)
Epoch: 4  Loss: tensor(1.4280, device='cuda:0', grad_fn=<NllLossBackward>)
Epoch: 5  Loss: tensor(1.3028, device='cuda:0', grad_fn=<NllLossBackward>)
Epoch: 6  Loss: tensor(1.1804, device='cuda:0', grad_fn=<NllLossBackward>)
Epoch: 7  Loss: tensor(1.0678, device='cuda:0', grad_fn=<NllLossBackward>)
Epoch: 8  Loss: tensor(0.9721, device='cuda:0', grad_fn=<NllLossBackward>)
Epoch: 9  Loss: tensor(0.8679, device='cuda:0', grad_fn=<NllLossBackward>)
Epoch: 10  Loss: tensor(0.7796, device='cuda:0', grad_fn=<NllLossBackward>)
Epoch: 11  Loss: tensor(0.7012, device='cuda:0', grad_fn=<NllLossBackward>)
Epoch: 12  Loss: tensor(0.6563, device='cuda:0', grad_fn=<NllLossBackward>)
Epoch: 13  Loss: tenso

In [16]:
model.eval()
_, pred = model(data).max(dim=1)
correct = int(pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
acc = correct / int(data.test_mask.sum())
print('Accuracy: {:.4f}'.format(acc))

Accuracy: 0.8080


### Batching (Minibatching do not need padding as giant Diagonal matrix is created)

In [49]:
from torch_geometric.datasets import TUDataset
from torch_geometric.data import DataLoader
from torch_scatter import scatter_mean

In [45]:
dataset = TUDataset(root='data', name='ENZYMES', use_node_attr=True)
loader = DataLoader(dataset, batch_size=32, shuffle=True)


Extracting data/ENZYMES/ENZYMES.zip
Processing...
Done!


In [52]:
for batch in loader:
    print(batch, batch.num_graphs) # Batch is column vector
    x = scatter_mean(batch.x, batch.batch, dim=0)
    print(x.size())

Batch(batch=[925], edge_index=[2, 3666], x=[925, 21], y=[32]) 32
torch.Size([32, 21])
Batch(batch=[1091], edge_index=[2, 4366], x=[1091, 21], y=[32]) 32
torch.Size([32, 21])
Batch(batch=[1109], edge_index=[2, 4322], x=[1109, 21], y=[32]) 32
torch.Size([32, 21])
Batch(batch=[954], edge_index=[2, 3734], x=[954, 21], y=[32]) 32
torch.Size([32, 21])
Batch(batch=[1176], edge_index=[2, 4502], x=[1176, 21], y=[32]) 32
torch.Size([32, 21])
Batch(batch=[1102], edge_index=[2, 4066], x=[1102, 21], y=[32]) 32
torch.Size([32, 21])
Batch(batch=[1072], edge_index=[2, 3960], x=[1072, 21], y=[32]) 32
torch.Size([32, 21])
Batch(batch=[1079], edge_index=[2, 3906], x=[1079, 21], y=[32]) 32
torch.Size([32, 21])
Batch(batch=[1024], edge_index=[2, 3866], x=[1024, 21], y=[32]) 32
torch.Size([32, 21])
Batch(batch=[977], edge_index=[2, 3390], x=[977, 21], y=[32]) 32
torch.Size([32, 21])
Batch(batch=[1095], edge_index=[2, 4138], x=[1095, 21], y=[32]) 32
torch.Size([32, 21])
Batch(batch=[857], edge_index=[2, 3404

### Transform

In [56]:
from torch_geometric.datasets import ShapeNet

dataset = ShapeNet(root='data', categories=['Airplane'])


Extracting data/shapenetcore_partanno_segmentation_benchmark_v0_normal.zip
Processing...
Done!


### Message Passing

In [1]:
import torch
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree


MessagePassing : Defines the aggregation to use (add,max,mean), direction of message passing  
MessagePassing.propagate() : start of message passing  
MessagePassing.message() : constructs the message(i:target or current node  j: neighbors)  
MessagePassing.udpate() : updates node embeddings

### KIPF(GCN) Message Passing

1. Add self-loops to the adjacency matrix.
2. Linearly transform node feature matrix.
3. Compute normalization coefficients.
4. Normalize node features in 𝜙.
5. Sum up neighboring node features ("add" aggregation).  
1-3 before message passing

In [11]:
class GCNConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super().__init__(aggr='add')  # "Add" aggregation (Step 5). Type of aggregation operation we want
        self.lin = torch.nn.Linear(in_channels, out_channels)

    def forward(self, x, edge_index):
        # x has shape [N, in_channels]
        # edge_index has shape [2, E]

        # Step 1: Add self-loops to the adjacency matrix.
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))

        # Step 2: Linearly transform node feature matrix.
        x = self.lin(x)

        # Step 3: Compute normalization.
        row, col = edge_index
        deg = degree(col, x.size(0), dtype=x.dtype)
        deg_inv_sqrt = deg.pow(-0.5)
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]     #(E,)

        # Step 4-5: Start propagating messages.
        return self.propagate(edge_index, x=x, norm=norm)       # Internally calls message(), aggregate() and update()

    def message(self, x_j, norm):
        # x_j has shape [E, out_channels]

        # Step 4: Normalize node features.
        return norm.view(-1, 1) * x_j                          # x_j neighboring node features of each node

In [13]:
#Calling the message Passing
conv = GCNConv(16, 32)
x = conv(x, edge_index)

NameError: name 'x' is not defined

### Edge Convolution

1. For each node i calculated diff between ith node representationa and each of its neighbor node(j) representation
2. Concat both node representations
3. Pass concat representation through Linear-ReLU-Linear layers
4. Max of all representation for each node i  
Edge convolution is dynamic convolution that recomputes the graph for each layer using Nearest neighbor in feature space

In [15]:
def EdgeConv(MessagePassing):
    
    def __init__(self, in_channels, out_channels):
        super().__init__(aggr='max')
        mlp = Seq(Linear(2*in_channels, out_channels),
                 ReLU(),
                 Linear(out_channels, out_channels))
        
    def forward(self, x, edge_index):
        self.propagate(edge_index, x=x)                 # x: (N, in_channels), edge_index: (2, E)
        
    def message(self, x_i, x_j):                        # x_i: (E, in_channels), x_j: (E, in_channels)
        tmp = torch.cat([x_i, x_j - x_i],dim=1)         #tmp: (E, 2*in_channels)
        return self.ml(tmp)